In [1]:
import re
import pandas as pd

## Exercise 1

Write a function named is_vowel. It should accept a string as input and use a regular expression to determine if the passed string is a vowel. While not explicity mentioned in the lesson, you can treat the result of re.search as a boolean value that indicates whether or not the regular expression matches the given string.



In [2]:
re.search(r"^(a|e|i|o|u)$", "a", re.IGNORECASE)

<re.Match object; span=(0, 1), match='a'>

In [3]:
re.search(r"^(a|e|i|o|u)$", "aeiou", re.IGNORECASE)

In [4]:
def is_vowel(string):
    return bool(re.search(r"^[aeiou]$", string, re.IGNORECASE))
    
assert is_vowel("a") == True
assert is_vowel("E") == True
assert is_vowel("aaa") == False
assert is_vowel("aeiou") == False

## Exercise 2

Write a function named is_valid_username that accepts a string as input. A valid username starts with a lowercase letter, and only consists of lowercase letters, numbers, or the _ character. It should also be no longer than 32 characters. The function should return either True or False depending on whether the passed string is a valid username.

In [5]:
# Why do we need the $ in the above pattern?
# because the regex below matches up to the capital letter but we're not saying everything needs to be lowercase
re.search(r"^[a-z][a-z0-9_]{,31}", "aaaCODEUPCODEUPaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaa")

<re.Match object; span=(0, 3), match='aaa'>

In [6]:
# starts with a lowercase letter
# is only lowercase letters, numbers, or _
# should be no longer than 32 characters
# return a boolean

def is_valid_username(string):
    pattern = r"^[a-z][a-z0-9_]{,31}$"
    return bool(re.search(pattern, string))

assert is_valid_username("codeup") == True
assert is_valid_username("codeup123") == True
assert is_valid_username("123Codeup") == False
assert is_valid_username("CodeupCodeup!") == False
assert is_valid_username("aaaCODEUPCODEUPaaaaaaaaaaaaaaaaaaaaaaaaaa") == False

## Exercise 3

Write a regular expression to capture phone numbers. It should match all of the following:

- (210) 867 5309
- +1 210.867.5309
- 867-5309
- 210-867-5309

#### Problem solving process:
- Put the subject strings in order of increasing complexity
- Solve them one at a time and build an iterative solution
- Add optionality as the pattern increases in parts

In [7]:
# Match seven digits with no other characters
re.search(r"\d{7}", "8675309")

<re.Match object; span=(0, 7), match='8675309'>

In [8]:
# Match 3 digits then a hypohen then 4 digits
re.search(r"\d{3}-\d{4}", "867-5309")

<re.Match object; span=(0, 8), match='867-5309'>

In [9]:
# Match 3 digits then a hypohen or a dot then 4 digits
re.search(r"\d{3}[-.]\d{4}", "867-5309")

<re.Match object; span=(0, 8), match='867-5309'>

In [10]:
# Match 3 digits then a hyphen or a dot then 4 digits
re.search(r"\d{3}[-.]\d{4}", "867.5309")

<re.Match object; span=(0, 8), match='867.5309'>

In [11]:
# Match 3 digits then a hyphen, dot, or space then 4 digits
re.search(r"\d{3}[-. ]\d{4}", "867 5309")

<re.Match object; span=(0, 8), match='867 5309'>

In [12]:
# Another approach on the delimiter could be to use \D for any non-digit
re.search(r"\d{3}\D?\d{4}", "8675309")

<re.Match object; span=(0, 7), match='8675309'>

In [13]:
re.search(r"\(?\d{3}\)?.?\d{3}.?\d{4}", "210-867-5309")

<re.Match object; span=(0, 12), match='210-867-5309'>

In [14]:
re.search(r"\(?\d{3}\)?.?\d{3}.?\d{4}", "210.867.5309")

<re.Match object; span=(0, 12), match='210.867.5309'>

In [15]:
re.search(r"(\(?\d{3}\)?)?.?\d{3}.?\d{4}", "(210) 867-5309")

<re.Match object; span=(0, 14), match='(210) 867-5309'>

In [16]:
re.search(r"(\(?\d{3}\)?)?.?\d{3}.?\d{4}", "867.5309")

<re.Match object; span=(0, 8), match='867.5309'>

In [17]:
# The regex here handles optional area code, optional parentheses, but looses the international code
# But what about the international code +1
re.search(r"(\(?\d{3}\)?)?.?\d{3}.?\d{4}", "+1 210.867.5309")

<re.Match object; span=(3, 15), match='210.867.5309'>

In [18]:
# make the calling code optional!
re.search(r"(\+\d+)?.?(\(?\d{3}\)?)?.?\d{3}.?\d{4}", "+1 210.867.5309")

<re.Match object; span=(0, 15), match='+1 210.867.5309'>

In [19]:
# make the calling code optional!
# make the + in the calling code optional
re.search(r"(\+?\d+)?.?(\(?\d{3}\)?)?.?\d{3}.?\d{4}", "+1 210.867.5309")

<re.Match object; span=(0, 15), match='+1 210.867.5309'>

## Exercise 3 using a DataFrame (and a different regex approach)

In [20]:
# another approach
# The \D*? means zero or more of anything that's not a digit (including parentheses)
# This is another way for specifying optional characters like literal "()" or "+"
phone_regex = re.compile(
"""
^
(?P<country_code>\+\d+)?
\D*?
(?P<area_code>\d{3})?
\D*?
(?P<exchange_code>\d{3})
\D*?
(?P<line_number>\d{4})
""", re.VERBOSE)

In [21]:
df = pd.DataFrame()
df['number'] = [
    '(210) 867 5309',
    '+1 210.867.5309',
    '867-5309',
    '210-867-5309',
    '2108675309',
]
df

,number
0,(210) 867 5309
1,+1 210.867.5309
2,867-5309
3,210-867-5309
4,2108675309


In [22]:
# extract turns named capture groups into dataframe columns
# NaNs for no match
df.number.str.extract(phone_regex)

,country_code,area_code,exchange_code,line_number
0,NaN,210,867,5309
1,+1,210,867,5309
2,NaN,NaN,867,5309
3,NaN,210,867,5309
4,NaN,210,867,5309


In [23]:
df = pd.concat([df, df.number.str.extract(phone_regex)], axis=1)
df

,number,country_code,area_code,exchange_code,line_number
0,(210) 867 5309,NaN,210,867,5309
1,+1 210.867.5309,+1,210,867,5309
2,867-5309,NaN,NaN,867,5309
3,210-867-5309,NaN,210,867,5309
4,2108675309,NaN,210,867,5309


## Exercise 4

Use regular expressions to convert the dates below to the standardized year-month-day format.

```
02/04/19
02/05/19
02/06/19
02/07/19
02/08/19
02/09/19
02/10/19
```

In [24]:
dates = [
    "02/04/19",
    "02/05/19",
    "02/06/19",
    "02/07/19",
    "02/08/19",
    "02/09/19",
    "02/10/19"
]

df = pd.DataFrame({"original": dates})
df

,original
0,02/04/19
1,02/05/19
2,02/06/19
3,02/07/19
4,02/08/19
5,02/09/19
6,02/10/19


In [25]:
pattern = re.compile(r"""
(?P<month>\d{2})/
(?P<day>\d{2})/
(?P<year>\d{2})
""", re.VERBOSE)

In [26]:
df = pd.concat([df, df.original.str.extract(pattern)], axis=1)
df

,original,month,day,year
0,02/04/19,02,04,19
1,02/05/19,02,05,19
2,02/06/19,02,06,19
3,02/07/19,02,07,19
4,02/08/19,02,08,19
5,02/09/19,02,09,19
6,02/10/19,02,10,19


In [27]:
df["new_format"] = df.year + "/" + df.month + "/" + df.day 
df

,original,month,day,year,new_format
0,02/04/19,02,04,19,19/02/04
1,02/05/19,02,05,19,19/02/05
2,02/06/19,02,06,19,19/02/06
3,02/07/19,02,07,19,19/02/07
4,02/08/19,02,08,19,19/02/08
5,02/09/19,02,09,19,19/02/09
6,02/10/19,02,10,19,19/02/10


## Exercise 5

Write a regex to extract the various parts of these logfile lines:


`GET /api/v1/sales?page=86 [16/Apr/2019:193452+0000] HTTP/1.1 {200} 510348 "python-requests/2.21.0" 97.105.19.58`

`POST /users_accounts/file-upload [16/Apr/2019:193452+0000] HTTP/1.1 {201} 42 "User-Agent: Mozilla/5.0 (X11; Fedora; Fedora; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/73.0.3683.86 Safari/537.36" 97.105.19.58`

`GET /api/v1/items?page=3 [16/Apr/2019:193453+0000] HTTP/1.1 {429} 3561 "python-requests/2.21.0" 97.105.19.58`

In [28]:
lines = [
    """GET /api/v1/sales?page=86 [16/Apr/2019:193452+0000] HTTP/1.1 {200} 510348 "python-requests/2.21.0" 97.105.19.58""",
    """POST /users_accounts/file-upload [16/Apr/2019:193452+0000] HTTP/1.1 {201} 42 "User-Agent: Mozilla/5.0 (X11; Fedora; Fedora; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/73.0.3683.86 Safari/537.36" 97.105.19.58""",
    """GET /api/v1/items?page=3 [16/Apr/2019:193453+0000] HTTP/1.1 {429} 3561 "python-requests/2.21.0" 97.105.19.58"""
]
lines

['GET /api/v1/sales?page=86 [16/Apr/2019:193452+0000] HTTP/1.1 {200} 510348 "python-requests/2.21.0" 97.105.19.58',
 'POST /users_accounts/file-upload [16/Apr/2019:193452+0000] HTTP/1.1 {201} 42 "User-Agent: Mozilla/5.0 (X11; Fedora; Fedora; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/73.0.3683.86 Safari/537.36" 97.105.19.58',
 'GET /api/v1/items?page=3 [16/Apr/2019:193453+0000] HTTP/1.1 {429} 3561 "python-requests/2.21.0" 97.105.19.58']

In [29]:
x = re.compile("""
\}\s(?P<bytes>\d+)\s\"
""", re.VERBOSE)
match = re.search(x, lines[0])
match

<re.Match object; span=(65, 75), match='} 510348 "'>

In [30]:
match.group("bytes")

'510348'

In [31]:
log_pattern = re.compile(r"""
(?P<method>GET|POST) 
\s
(?P<path>/[/\w\-\?=]+)
\s
\[(?P<timestamp>.+)\]
\s
(?P<http_version>HTTP/\d+\.\d+)
\s
\{(?P<status_code>\d+)\}
\s
(?P<bytes>\d+)
\s
"(?P<user_agent>.+)"
\s
(?P<ip>\d{1,3}\.\d{1,3}\.\d{1,3}\.\d{1,3})
$
""", re.VERBOSE)

In [32]:
rows = [re.search(log_pattern, line).groupdict() for line in lines]
rows

[{'method': 'GET',
  'path': '/api/v1/sales?page=86',
  'timestamp': '16/Apr/2019:193452+0000',
  'http_version': 'HTTP/1.1',
  'status_code': '200',
  'bytes': '510348',
  'user_agent': 'python-requests/2.21.0',
  'ip': '97.105.19.58'},
 {'method': 'POST',
  'path': '/users_accounts/file-upload',
  'timestamp': '16/Apr/2019:193452+0000',
  'http_version': 'HTTP/1.1',
  'status_code': '201',
  'bytes': '42',
  'user_agent': 'User-Agent: Mozilla/5.0 (X11; Fedora; Fedora; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/73.0.3683.86 Safari/537.36',
  'ip': '97.105.19.58'},
 {'method': 'GET',
  'path': '/api/v1/items?page=3',
  'timestamp': '16/Apr/2019:193453+0000',
  'http_version': 'HTTP/1.1',
  'status_code': '429',
  'bytes': '3561',
  'user_agent': 'python-requests/2.21.0',
  'ip': '97.105.19.58'}]

In [33]:
df = pd.DataFrame(rows)
df

,method,path,timestamp,http_version,status_code,bytes,user_agent,ip
0,GET,/api/v1/sales?page=86,16/Apr/2019:193452+0000,HTTP/1.1,200,510348,python-requests/2.21.0,97.105.19.58
1,POST,/users_accounts/file-upload,16/Apr/2019:193452+0000,HTTP/1.1,201,42,User-Agent: Mozilla/5.0 (X11; Fedora; Fedora; ...,97.105.19.58
2,GET,/api/v1/items?page=3,16/Apr/2019:193453+0000,HTTP/1.1,429,3561,python-requests/2.21.0,97.105.19.58


In [34]:
# let's make a column that is a boolean whether or not the path contained a query string
# simple answer on one string: look for a question mark character
def is_query_string(string):
    return "?" in string

In [35]:
# df["has_query_string"] = df["path"].apply(lambda string: "?" in string)
df["has_query_string"] = df["path"].apply(is_query_string)
df

,method,path,timestamp,http_version,status_code,bytes,user_agent,ip,has_query_string
0,GET,/api/v1/sales?page=86,16/Apr/2019:193452+0000,HTTP/1.1,200,510348,python-requests/2.21.0,97.105.19.58,True
1,POST,/users_accounts/file-upload,16/Apr/2019:193452+0000,HTTP/1.1,201,42,User-Agent: Mozilla/5.0 (X11; Fedora; Fedora; ...,97.105.19.58,False
2,GET,/api/v1/items?page=3,16/Apr/2019:193453+0000,HTTP/1.1,429,3561,python-requests/2.21.0,97.105.19.58,True


## Bonus Exercise
You can find a list of words on your mac at `/usr/share/dict/words`. Use this file to answer the following questions:

- How many words have at least 3 vowels?
- How many words have at least 3 vowels in a row?
- How many words have at least 4 consonants in a row?
- How many words start and end with the same letter?
- How many words start and end with a vowel?
- How many words contain the same letter 3 times in a row?
- What other interesting patterns in words can you find?

In [36]:
df = pd.read_csv("/usr/share/dict/words", header=None)
df.columns = ["word"]
df.head()

,word
0,A
1,a
2,aa
3,aal
4,aalii


In [37]:
three_or_more_vowels = df.word.str.count(r"[aeiouAEIOU]") >= 3

In [38]:
# How many words have at least 3 vowels
df[three_or_more_vowels]

,word
4,aalii
6,Aani
7,aardvark
8,aardwolf
9,Aaron
...,...
235874,zymotically
235875,zymotize
235876,zymotoxic
235878,Zyrenian


In [39]:
# How many words have at least 3 vowels in a row?
df[df.word.str.count(r"[aeiouAEIOU]{3}") > 0]

,word
234,Abietineae
235,abietineous
301,ablatitious
434,abranchious
507,absenteeism
...,...
235800,Zygophyceae
235801,zygophyceous
235802,Zygophyllaceae
235803,zygophyllaceous
